In [7]:
!pip install --upgrade openai

import openai
import os
import time

openai.api_key = os.getenv("OPENAI_API_KEY")

def query_messi_info(query):

    """
    Retrieves information from a local file and queries GPT for an answer.
    Includes retry logic for rate limits and temporary errors.
    """

    if not os.path.exists("messi.txt"):
        with open("messi.txt", "w") as file:
            file.write("Lionel Messi is an Argentinian football player.")

    # Read file content
    with open("messi.txt", "r") as file:
        messi_info = file.read()

    # If no API key, return a mock answer (for demo purposes)
    if not openai.api_key:
        return f"[MOCK RESPONSE] Based on the file: {messi_info}\nMessi has won numerous Ballon d'Or awards, multiple UEFA Champions League titles, and the 2022 FIFA World Cup with Argentina."

    # Retry parameters
    max_retries = 5
    retry_delay = 2

    for i in range(max_retries):
        try:
            response = openai.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": f"Given the following information about Lionel Messi: {messi_info}"},
                    {"role": "user", "content": query},
                ],
                max_tokens=150
            )
            return response.choices[0].message.content.strip()

        except openai.RateLimitError:
            if i < max_retries - 1:
                time.sleep(retry_delay)
                retry_delay *= 2
                print(f"Rate limit hit. Retrying in {retry_delay} seconds...")
            else:
                return "Failed due to rate limits."
        except Exception as e:
            return f"Request failed: {e}"


if __name__ == "__main__":
  user_query = "What are Lionel Messi's major achievements?"
  answer = query_messi_info(user_query)

  print(answer)

[MOCK RESPONSE] Based on the file: Lionel Messi is an Argentinian football player.
Messi has won numerous Ballon d'Or awards, multiple UEFA Champions League titles, and the 2022 FIFA World Cup with Argentina.
